In [1]:
import alborexdata
import netCDF4
import glob
import os
import json
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap 
import numpy as np
import datetime
import cmocean
from importlib import reload
import urllib.request

/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
reload(alborexdata)

<module 'alborexdata' from '/home/ctroupin/Publis/201703_AlborexData/python/alborexdata.py'>

In [3]:
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# Configuration

In [13]:
with open('alborexconfig.json') as json_data_file:
    config = json.load(json_data_file)

In [5]:
makeFigs, makeFigsLeaflet = 1, 0

Create a logger

In [6]:
logger = alborexdata.configure_logging("./logs/alborexFigure2.log")

Domain

In [7]:
coordinates = config["domain"]["coordinates1"]
coordinates2 = config["domain"]["coordinates2"]

Create a list of netCDF files containing SST

In [8]:
coastfile = config["datafiles"]["coast"]
sstfilelist = config["datafiles"]["sst"]
sstfilelistremote = config["datafiles"]["sstremote"]
for sstfile, sstremote in zip(sstfilelist, sstfilelistremote):
    if not(os.path.exists(sstfile)):
        logger.info("Downloading data file {}".format(os.path.basename(sstfile)))
        urllib.request.urlretrieve(sstremote, sstfile)
    else:
        logger.info("SST file {} already downloaded".format(os.path.basename(sstfile)))

2019-01-09 16:06:24,070 - alborex_logger - INFO - SST file A2014145125000.L2_LAC_SST.nc already downloaded
2019-01-09 16:06:24,072 - alborex_logger - INFO - SST file A2014150020500.L2_LAC_SST.nc already downloaded


In [9]:
cmap = plt.cm.RdYlBu_r

# Data reading and plotting

Make figure directory if necessary.

In [10]:
if os.path.isdir(config["figdir"]):
    logger.debug("Figure directory {0} already exists".format(config["figdir"]))
else:
    os.makedirs(config["figdir"])
    logger.debug("Creating figure directory {0}".format(config["figdir"]))
    
if os.path.isdir(config["figdirleaflet"]):
    logger.debug("Figure directory {0} already exists".format(config["figdirleaflet"]))
else:
    os.makedirs(config["figdirleaflet"])
    logger.debug("Creating figure directory {0}".format(config["figdirleaflet"]))

2019-01-09 16:06:27,952 - alborex_logger - DEBUG - Figure directory ../figures already exists
2019-01-09 16:06:27,957 - alborex_logger - DEBUG - Figure directory ../leaflet/images/ already exists


## Coastline

In [11]:
loncoast, latcoast = alborexdata.read_lonlat_coast(config["datafiles"]["coast"])

Create the projection (only once)

In [12]:
m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='h')

Projection for overlay in `leaflet`

In [ ]:
if makeFigsLeaflet:
    m2 = Basemap(llcrnrlon=coordinates[0],
         llcrnrlat=coordinates[2],
         urcrnrlon=coordinates[1],
         urcrnrlat=coordinates[3], resolution = 'l', epsg=3857)

## Front position

In [14]:
frontcoords = config["datafiles"]["front"]
f = alborexdata.Front()
f.get_from_file(frontcoords)
f.smooth()

# Main plot

In [16]:
# Rectangle for the region of interest
xrect = (coordinates2[0], coordinates2[1], coordinates2[1], coordinates2[0], coordinates2[0])
yrect = (coordinates2[2], coordinates2[2], coordinates2[3], coordinates2[3], coordinates2[2])
    
figname = "fig02.png"
fig = plt.figure(figsize=(10, 10))

for ii, sstfiles in enumerate(sstfilelist):
    logger.info("Working on file: {0}".format(os.path.basename(sstfiles)))
    
    # Read data from file
    lon, lat, sst, sstqual, year, day, platform = alborexdata.load_sst_l2(sstfiles)
    # Make the SST when flag > 1
    sst = np.ma.masked_where(sstqual > 1, sst)
    
    # Create date from year and day
    titledate = (datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(day) - 1)).strftime('%Y-%m-%d')
    figtitle = ' '.join(('MODIS', platform, '$-$', titledate))
    
    
    ax = plt.subplot(2,1,ii+1)
    m.ax = ax
    pcm = m.pcolormesh(lon, lat, sst, latlon=True, 
                       vmin=18., vmax=20.5, cmap=cmap)

    # Add the coastline
    for i in range(0, len(loncoast)):
        m.plot(np.array(loncoast[i]), np.array(latcoast[i]), 
               color='k', linewidth=.25, latlon=True)

    alborexdata.add_map_grid(m, coordinates, dlon=2., dlat=2.,
                             fontname='Times New Roman', fontsize=14, 
                             linewidth=0.2, zorder=1, color=".6")

    m.drawmapscale(-.75, 35.25, -1, 35.25, 75, fontsize=12)

    m.plot(xrect, yrect, ":", color='k', latlon=True)

    xtext1, ytext1 = m(-3.2, 37.)
    xfig1, yfig1 = m(coordinates2[0], coordinates2[2])

    # Plot front
    m.plot(f.lon, f.lat, "k--", linewidth=2, latlon=True)
    ax.set_title(figtitle, fontsize=20)

fig.subplots_adjust(right=0.95)
cbar_ax = fig.add_axes([0.85, 0.15, 0.035, 0.7])
cb = plt.colorbar(pcm, cax=cbar_ax, extend='both')
cb.set_label('$^{\circ}$C', rotation=0, ha='left', fontsize=16)
cb.ax.tick_params(labelsize=14)

# plt.show()
plt.savefig(os.path.join(config["figdir"], figname), dpi=300, bbox_inches='tight')
plt.close()

2019-01-09 16:09:25,558 - alborex_logger - INFO - Working on file: A2014145125000.L2_LAC_SST.nc
2019-01-09 16:09:26,629 - alborex_logger - INFO - Working on file: A2014150020500.L2_LAC_SST.nc


## Loop on the files

In [17]:
for sstfiles in sstfilelist:
    logger.info("Working on file: {0}".format(os.path.basename(sstfiles)))
    
    # Read data from file
    lon, lat, sst, sstqual, year, day, platform = alborexdata.load_sst_l2(sstfiles)
    
    # Create date from year and day
    titledate = (datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(day) - 1)).strftime('%Y-%m-%d')
    figtitle = ' '.join(('MODIS', platform, '$-$', titledate))
    # logger.debug(title)
    
    # Make the SST when flag > 1
    sst = np.ma.masked_where(sstqual > 1, sst)

    # Create figure name (remove .nc extension and substitute . by _)
    figname = ''.join(('_'.join(os.path.basename(sstfiles).split('.')[:-1]), '_cm.png'))
    logger.info("Making figure {0}".format(figname))
    
    # Rectangle for the region of interest
    xrect = (coordinates2[0], coordinates2[1], coordinates2[1], coordinates2[0], coordinates2[0])
    yrect = (coordinates2[2], coordinates2[2], coordinates2[3], coordinates2[3], coordinates2[2])
    
    if makeFigs:
        # Normal figures
        fig = plt.figure(figsize=(10, 10))
        ax = plt.subplot(111)
        m.ax = ax
        pcm = m.pcolormesh(lon, lat, sst, latlon=True, 
                           vmin=18., vmax=20.5, cmap=cmocean.cm.thermal)

        # Add the contours
        for i in range(0, len(loncoast)):
            m.plot(np.array(loncoast[i]), np.array(latcoast[i]), 
                   color='k', linewidth=.25, latlon=True)

        alborexdata.add_map_grid(m, coordinates, dlon=2., dlat=2.,
                                 fontname='Times New Roman', fontsize=14, 
                                 linewidth=0.2, zorder=1, color=".6")

        #m.fillcontinents(color='w', zorder=2)
        #ax.set_xlim(coordinates[0], coordinates[1])
        #ax.set_ylim(coordinates[2], coordinates[3])

        cb = plt.colorbar(pcm, extend='both', shrink=0.6)
        cb.set_label('$^{\circ}$C', rotation=0, ha='left', fontsize=16)
        m.drawmapscale(-.75, 35.25, -1, 35.25, 75, fontsize=12)
        
        m.plot(xrect, yrect, ":", color='.25', latlon=True)

        xtext1, ytext1 = m(-3.2, 37.)
        xfig1, yfig1 = m(coordinates2[0], coordinates2[2])
        
        # Plot front
        m.plot(f.lon, f.lat, "k--", linewidth=2, latlon=True)
              
        plt.title(figtitle, fontsize=20)
        plt.savefig(os.path.join(config["figdir"], figname), dpi=300, bbox_inches='tight')
        plt.close()
        
    if makeFigsLeaflet:
    
        # Figures without border and axes
        llon, llat = m2(lon, lat)
        fig = plt.figure(frameon=False)
        ax = fig.add_axes([0, 0, 1, 1])
        m2.pcolormesh(llon, llat, sst, vmin=17., vmax=20., cmap=cmap)
        ax.axis('off')
        #ax.set_xlim(lon.min(), lon.max())
        #ax.set_ylim(lat.min(), lat.max())
        f1 = plt.gca()
        f1.axes.get_xaxis().set_ticks([])
        f1.axes.get_yaxis().set_ticks([])
        plt.savefig(os.path.join(config["figdirleaflet"], figname), transparent=True, 
                    bbox_inches='tight', pad_inches=0)
        plt.close()

2019-01-09 16:09:54,718 - alborex_logger - INFO - Working on file: A2014145125000.L2_LAC_SST.nc
2019-01-09 16:09:55,333 - alborex_logger - INFO - Making figure A2014145125000_L2_LAC_SST_cm.png
2019-01-09 16:09:59,396 - alborex_logger - INFO - Working on file: A2014150020500.L2_LAC_SST.nc
2019-01-09 16:09:59,666 - alborex_logger - INFO - Making figure A2014150020500_L2_LAC_SST_cm.png
